# Import Dependencies

In [ ]:
!pip install music21
#Importing Libraries
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

# Loading Data

In [ ]:
import os
from tqdm.notebook import tqdm
#Loading the list of chopin's midi files as stream 
filepath = "../input/classical-music-midi/chopin/"
#Getting midi files
all_midis= []
for i in tqdm(os.listdir(filepath)):
    if i.endswith(".mid"):
        tr = filepath+i
        midi = converter.parse(tr)
        all_midis.append(midi)
        

## get the components out of these midi files

In [ ]:
def extract_feature(file):
    notes = []
    pick = None
    for j in tqdm(file, desc='Processing Files', unit='file'):
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))
                    
    return notes


# Getting the list of notes as corpus

In [ ]:
Corpus = extract_feature(all_midis)
print('Total notes in all the Chopin midis in the dataset:', len(Corpus))

# Data Exploration

In [ ]:
print(Corpus[:50])

**Printing Music Sheet**

In [ ]:
#to play audio or corpus
print("Sample Audio From Data")
IPython.display.Audio("../input/music-generated-lstm/Corpus_Snippet.wav") 

# Examine all the notes in the  corpus

In [ ]:
from collections import Counter

count_num = Counter(Corpus)

print("Total unique notes in the Corpus:", len(count_num))


In [ ]:
# Exploring the notes dictionary
Notes = list(count_num.keys())
Recurrence = list(count_num.values())
# Average recurrent for a note in Corpus
def Average(lis):
    return sum(lis) / len(lis)
print('Average recurrence for a note in corpus:', Average(Recurrence))
print('Most frequent note in corpus appeared:', max(Recurrence), 'times')
print('Least frequent note in corpus appeared:', min(Recurrence), 'times')

# Ploting the distribution of notes

In [ ]:
plt.figure(figsize=(18, 3), facecolor='#97BACB')
bins = np.arange(0,(max(Recurrence)), 50)
plt.hist(Recurrence, bins=bins, color='#97BACB')
plt.axvline(x=100, color='#DBACC1')
plt.title('Frequency Distribution of Notes in the Corpus')
plt.xlabel('Frequency Of The Cords in Corpus')
plt.ylabel('Number Of Chords')
plt.show()

In [ ]:
# taking note which played less than 100 times
rare_note = []
for index, (key, value) in enumerate(count_num.items()):
    if value < 100 :
        m = key
        rare_note.append(m)
        
print('Total number of notes that occur less then 100 times:', len(rare_note))

In [ ]:
# Eleminating the rare note
for element in Corpus:
    if element in rare_note:
        Corpus.remove(element)
        
print('Length of corpus after elemination the rare notes', len(Corpus))

# Data Preprocessing

In [ ]:
symb=sorted(list(set(Corpus)))
L_corpus =len(Corpus) # length of corpus
L_symb = len(symb) # Length of total unique character
# Building dictonary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i,c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print('Number of unique characters:', L_symb)

In [ ]:
# Encoding and Spliting dataset
length = 40
features = []
targets = []
for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for  j in feature])
    targets.append(mapping[target])
    
L_datapoints = len(targets)
print('Total number of sequences in the corpus:', L_datapoints)

In [ ]:
X = (np.reshape(features, (L_datapoints, length, 1))) / float(L_symb)
y = tensorflow.keras.utils.to_categorical(targets)

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=.2)

# Model Architecture

In [ ]:
from tensorflow.keras.layers import Bidirectional
model = Sequential()
model.add(Bidirectional(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))

opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)
model.build(input_shape=(None, X.shape[1], X.shape[2]))

In [ ]:
model.summary()

In [ ]:
history = model.fit(xtrain,ytrain,batch_size=256, epochs=200)

# Function To Generate Music

In [ ]:
def Malody_Generator(Note_Count):
    seed = xtest[np.random.randint(0,len(xtest)-1)]
    Music = ""
    Notes_Generated=[]
    for i in range(Note_Count):
        seed = seed.reshape(1,length,1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction) / 1.0 #diversity
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        index_N = index/ float(L_symb)   
        Notes_Generated.append(index)
        Music = [reverse_mapping[char] for char in Notes_Generated]
        seed = np.insert(seed[0],len(seed[0]),index_N)
        seed = seed[1:]
    #Now, we have music in form or a list of chords and notes and we want to be a midi file.
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)   
    return Music,Melody_midi


#getting the Notes and Melody created by the model
Music_notes, Melody = Malody_Generator(100)

**Sample Music**

In [ ]:
Melody.write('midi', 'Melody_Generated.mid')
IPython.display.Audio('../input/music-generated-lstm/Melody_Generated 2.wav')